<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---



# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

O objetivo deste projeto é realizar uma análise exploratória dos dados fornecidos pela Loggi BUD, com o intuito de identificar insights que possam otimizar o processo logístico de entregas em Brasília, DF. Através de métodos de análise e visualização de dados, buscamos propor melhorias que possam beneficiar a eficiência e a experiência do cliente.

## 2\. Pacotes e bibliotecas

In [ ]:
# Instalando e importando as bibliotecas que serão utilizadas no projeto

!pip3 install geopandas;

import json

import pandas as pd
import geopandas
import seaborn as sns
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import matplotlib.pyplot as plt

## 3\. Exploração de dados

É feita e extração do dado bruto e criado um arquivo chamado deliveries.json



In [ ]:
!wget -q "http://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

# Os dados do arquivo são carregados em uma variável do tipo dicionário chamada 'data'
with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

# Transformando o dicionário em um DataFrame
deliveries_df = pd.DataFrame(data)

#Visualização dos dados
deliveries_df.head()

A coluna origin possui dados aninhados no formato JSON, portanto é feita a operação de achatamento para adiciona-la ao DataFrame principal.

In [ ]:
# É feito o achatamento da coluna 'origin', criando um novo DataFrame chamado hub_origin_df
hub_origin_df = pd.json_normalize(deliveries_df['origin'])

# É feita a junção de ambos os DataFrames através do índice
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

# A coluna 'origin' é descartada, e a ordem das colunas do DataFrame é organizada
deliveries_df = deliveries_df.drop('origin', axis=1)
deliveries_df = deliveries_df[['name', 'region', 'lng', 'lat', 'vehicle_capacity', 'deliveries']]

# As colunas são renomeadas para ter mais clareza
deliveries_df.rename(columns={'lng': 'hub_lng', 'lat': 'hub_lat'}, inplace=True)

# Visualização dos dados
deliveries_df.head()

A coluna deliveries contém uma lista de dados aninhados, serão feitos os processos de explosão e achatamento, e enfim os dados serão adicionados ao DataFrame principal.

In [ ]:
# É realizado o processo de explosão na coluna 'deliveries', criando o DataFrame deliveries_exploded_df
deliveries_exploded_df = deliveries_df[['deliveries']].explode('deliveries')

# É criado um novo DataFrame, organizando os dados que serão agregados ao DataFrame principal
deliveries_normalized_df = pd.concat([
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda record: record['size'])).rename(columns={'deliveries': 'delivery_size'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda record: record['point']['lng'])).rename(columns={'deliveries': 'delivery_lng'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda record: record['point']['lat'])).rename(columns={'deliveries': 'delivery_lat'}),
], axis= 1)

# A coluna 'deliveries' é descartada, e os dados agregados em um único DataFrame
deliveries_df = deliveries_df.drop('deliveries', axis = 1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)

# Visualização dos dados
deliveries_df.head()

## 3.1\. Verificação da estrutura do DataFrame

In [ ]:
deliveries_df.shape

In [ ]:
deliveries_df.columns

In [ ]:
deliveries_df.index

In [ ]:
deliveries_df.info()

In [ ]:
deliveries_df.dtypes

O DataFrame principal contém um schema consiso e não possui valores nulos, portanto podemos prosseguir com a manipulação, enriquecimento e visualização dos dados.

## 4\. Manipulação

Primeiramente, será realizada a geocodificação reversa dos hubs, adicionando suas localizações ao DataFrame.

In [ ]:
# Para o processo de geocodificação reversa dos hubs, colocamos seus dados em um DataFrame separado
hub_df = deliveries_df[['region', 'hub_lng', 'hub_lat']]
hub_df = hub_df.drop_duplicates().sort_values(by='region').reset_index(drop=True)

geolocator = Nominatim(user_agent='ebac_geocoder')
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Os dados de latitude e longitude são concatenados, e é feita a geocodificação
hub_df['coordinates'] = hub_df['hub_lat'].astype(str) + ', ' + hub_df['hub_lng'].astype(str)
hub_df['geodata'] = hub_df['coordinates'].apply(geocoder)

# Visualização dos dados
hub_df.head()

Os dados na coluna 'geodata' estão em uma lista JSON, portanto iremos normalizar esta coluna, filtrar os dados necessários e adicioná-los ao DataFrame principal.

In [ ]:
# É feito o processo de normalização da coluna 'geodata'
hub_geodata_df = pd.json_normalize(hub_df['geodata'].apply(lambda data: data.raw))

# Os dados necessários são filtrados e propriamente nomeados para que o DataFrame possa ser lido com clareza
hub_geodata_df = hub_geodata_df[['address.town', 'address.suburb', 'address.city']]
hub_geodata_df.rename(columns={'address.town': 'hub_town', 'address.suburb': 'hub_suburb', 'address.city': 'hub_city'}, inplace=True)
hub_geodata_df['hub_city'] = np.where(hub_geodata_df['hub_city'].notna(), hub_geodata_df['hub_city'], hub_geodata_df['hub_town'])
hub_geodata_df['hub_suburb'] = np.where(hub_geodata_df['hub_suburb'].notna(), hub_geodata_df['hub_suburb'], hub_geodata_df['hub_city'])
hub_geodata_df = hub_geodata_df.drop('hub_town', axis=1)

# Os dados filtrados são adicionados ao DataFrame hub_df, e finalmente é feita a junção com o DataFrame principal, organizando as colunas
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[['region', 'hub_suburb', 'hub_city']]
deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how='inner', on='region')
deliveries_df = deliveries_df[['name', 'region', 'hub_lng', 'hub_lat', 'hub_city', 'hub_suburb', 'vehicle_capacity', 'delivery_size', 'delivery_lng', 'delivery_lat']]

# Visualização dos dados
deliveries_df.head()

Agora será feita a geocodificação reversa dos endereços de cada entrega, utilizando o arquivo csv providenciado pela EBAC.

In [ ]:
# Extração do dado bruto no arquivo csv fornecido
!wget -q 'https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv' -O deliveries-geodata.csv

# Visualização dos dados
deliveries_geodata_df = pd.read_csv('deliveries-geodata.csv')
deliveries_geodata_df.head()

In [ ]:
# Junção dos dados do arquivo csv com o DataFrame principal
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[['delivery_city', 'delivery_suburb']], how='inner', left_index=True, right_index=True)
deliveries_df.head()

## 4.1\. Verificação da estrutura e qualidade do DataFrame

Verificação de valores nulos.

In [ ]:
deliveries_df.info()

Confirmação de que apenas as colunas 'delivery_city' e 'delivery_suburb' possuem valores nulos.

In [ ]:
deliveries_df.isna().any()

Porcentagem dos dados nulos na coluna 'delivery_city'

In [ ]:
100 * (deliveries_df['delivery_city'].isna().sum() / len(deliveries_df))

Porcentagem de dados nulos na coluna 'delivery_suburb'

In [ ]:
100 * (deliveries_df['delivery_suburb'].isna().sum() / len(deliveries_df))

In [ ]:
prop_df = deliveries_df[['delivery_city']].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

In [ ]:
prop_df = deliveries_df[['delivery_suburb']].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

**Conclusão:** há uma notável quandidade de valores nulos na coluna delivery_suburb, assim como Brasília sendo a cidade e bairro mais frequente, mesmo não sendo nenhum dos dois. Portanto, devido à estas discrepâncias, podemos presumir que os valores desta coluna não serão tão úteis para análise.

## 5\. Visualização

Para realizar a visualização dos dados obtidos até agora, será feito o download de mapas do Distrito Federal diretamente do site oficial do IBGE.

In [ ]:
# Download dos mapas necessários
!wget -q "https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip" -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

In [ ]:
# Gerando a localização do Distrito Federal utilizando os arquivos
mapa = geopandas.read_file("distrito-federal.shp")
mapa = mapa.loc[[0]]

# Visualização dos dados
mapa.head()

In [ ]:
# Com os arquivos, criaremos o DataFrame geo_hub_df, e a localização dos hubs estará na coluna 'geometry'
hub_df = deliveries_df[['region', 'hub_lng', 'hub_lat']].drop_duplicates().reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame(hub_df, geometry=geopandas.points_from_xy(hub_df['hub_lng'], hub_df['hub_lat']))

# Visualização dos dados
geo_hub_df.head()

O mesmo processo será feito para a localização das entregas.

In [ ]:
geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df, geometry=geopandas.points_from_xy(deliveries_df['delivery_lng'], deliveries_df['delivery_lat']))

# Visualização dos dados
geo_deliveries_df.head()

Utilizando o pacote
 matplotlib, será criada a vizualização dos hubs e suas respectivas entregas.

In [ ]:
# É criado o plot vazio
fi, ax = plt.subplots(figsize = (50/2.54, 50/2.54))

# É criado o plot do mapa do Distrito Federal
mapa.plot(ax=ax, alpha=0.4, color='lightgrey')

# Criação do plot dos deliveries
geo_deliveries_df.query('region == "df-0"').plot(ax=ax, markersize=1, color='red', label='df-0')
geo_deliveries_df.query('region == "df-1"').plot(ax=ax, markersize=1, color='blue', label='df-1')
geo_deliveries_df.query('region == "df-2"').plot(ax=ax, markersize=1, color='seagreen', label='df-2')

# Plot dos hubs
geo_hub_df.plot(ax=ax, markersize=30, marker='x', color='black', label='hub')

# Plot da legenda
plt.title('Entregas no Distrito Federal por Região', fontdict={'fontsize': 16})
lgnd = plt.legend(prop={'size': 15})
for handle in lgnd.legendHandles:
  handle.set_sizes([50])

Proporção de entregas de cada hub no Distrito Federal.

In [ ]:
data = pd.DataFrame(deliveries_df[['region', 'vehicle_capacity']].value_counts(normalize=True)).reset_index()
data.rename(columns={'proportion': 'region_percent'}, inplace=True)

# Visualização dos dados
data.head()

In [ ]:
# Utilizando o pacote Seaborn, será criado um gráfico representando a proporção de entregas de cada hub

with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=data, x='region', y='region_percent', errorbar=None, palette='pastel')
  grafico.set(title='Proporção de entregas por região', xlabel='Região', ylabel='Proporção');

## 6\. Conclusão / Insights

Após a realização do processo de análise exploratória de dados, e com a visualização do mapa e do grafico que foram gerados, podemos chegar aos seguintes insights:


1.   As entregas de cada hub estão corretamente alocadas, e difícilmente entraram em conflito com outro hub;
2.   Os hubs 0 e 2 têm que fazer entregas distantes, o que pode gerar aumento no tempo e preço de entrega, assim como afetar a manutenção dos veículos;
3.   Juntos os hubs 1 e 2 estão encarregados de 89% das entregas. Como todos os veículos tem a mesma capacidade, pode-se pensar na possibilidade de deslocar alguns veículos do hub 0 para regiões de mais tráfego.

Junto com estes insights, pode-se sugerir a possibilidade de realocação ou até mesmo criação de um novo hub para atender regiões mais distantes.